In [1]:
import boto3
import pandas as pd

In [2]:
import sys
print(sys.path)

['/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/data_preproc/notebooks', '/Users/dariamishina/.conda/envs/untitled/lib/python38.zip', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/lib-dynload', '', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/site-packages']


In [3]:
# добавляем в path папку с тем файлом, откуда надо импортировать функции
# sys.path.append('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/data_preproc/src')
# sys.path.append('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/data_preproc')
sys.path.append('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

In [4]:
from data_preproc.src.utils import preprocess_text1, preprocess_text2, preprocess_text3, preprocess_text4, tokenizing

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
# sys.path.remove('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

# загружаем из s3

In [5]:
session = boto3.session.Session()

In [6]:
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='REDACTED_KEY_ID_1',
    aws_secret_access_key='REDACTED_SECRET_KEY_1',
    region_name='ru-cental1'
)

In [10]:
BUCKET_NAME = "mlds23-authorship-identification"
BUCKET_DIR = "splitted_data/"
FILENAME = 'splitted_df.csv'
#загружает в локальную директорию, потом отдельно надо считывать
s3.download_file(Filename=FILENAME, Bucket=BUCKET_NAME, Key=BUCKET_DIR + FILENAME)

# делаем предобработку 

In [7]:
df = pd.read_csv('splitted_df.csv')

In [8]:
#у всех функций параметры по умолчанию: text, tokenize = True, tostr=False. Для tfidf tostr=True
df['text_preproc1'] = df['text'].apply(lambda x: preprocess_text1(x, tostr=True))
df['text_preproc2'] = df['text'].apply(lambda x: preprocess_text2(x, tostr=True))
df['text_preproc3'] = df['text'].apply(lambda x: preprocess_text3(x, tostr=True))
df['text_preproc4'] = df['text'].apply(lambda x: preprocess_text4(x, tostr=True))

In [9]:
df.to_csv('preproc_df.csv', index=False)

# загружаем в s3

In [13]:
DATA_PATH = '/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/data_preproc/notebooks/' #это локальный путь! Куда сохранился мой csv файл
NEW_FILE_NAME = 'preproc_df.csv'

In [14]:
s3.upload_file(f'{DATA_PATH}{NEW_FILE_NAME}', BUCKET_NAME, f'preproc_data/{NEW_FILE_NAME}')